# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Set Path

In [2]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Load Data

In [3]:
df_train = pd.read_pickle(str(data_dir / "gsdc_cleaned_train.pkl.gzip"))

In [5]:
df_test = pd.read_pickle(str(data_dir / "gsdc_cleaned_test.pkl.gzip"))

In [6]:
for col in df_train.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
TimeNanos
LeapSecond
FullBiasNanos
BiasNanos
BiasUncertaintyNanos
DriftNanosPerSecond
DriftUncertaintyNanosPerSecond
HardwareClockDiscontinuityCount
Svid
TimeOffsetNanos
State
ReceivedSvTimeNanos
ReceivedSvTimeUncertaintyNanos
Cn0DbHz
PseudorangeRateMetersPerSecond
PseudorangeRateUncertaintyMetersPerSecond
AccumulatedDeltaRangeState
AccumulatedDeltaRangeMeters
AccumulatedDeltaRangeUncertaintyMeters
CarrierFrequencyHz
MultipathIndicator
ConstellationType
AgcDb
BasebandCn0DbHz
FullInterSignalBiasNanos
FullInterSignalBiasUncertaintyNanos
SatelliteInterSignalBiasNanos
SatelliteInterSignalBiasUnc

# Extract Data

## Get dlatDeg, dlngDeg, dheightAboveWgs84EllipsoidM

In [64]:
def getDeltaPositions(df:pd.DataFrame)->pd.DataFrame:
    output = df.copy()
    phones = output['phone'].unique()
    output[['dlatDeg', 'dlngDeg', 'dheight']] = 0
    for phone in phones:
        latDeg = output.loc[output['phone'] == phone, 'latDeg']
        output.loc[output['phone'] == phone, 'dlatDeg'] = latDeg - latDeg.iloc[0]
        
        lngDeg = output.loc[output['phone'] == phone, 'lngDeg']
        output.loc[output['phone'] == phone, 'dlngDeg'] = lngDeg - lngDeg.iloc[0]
        
        heightAboveWgs84EllipsoidM = output.loc[output['phone'] == phone, 'heightAboveWgs84EllipsoidM']
        output.loc[output['phone'] == phone, 'dheight'] = heightAboveWgs84EllipsoidM - heightAboveWgs84EllipsoidM.iloc[0]
        
    return output

In [65]:
df_train = getDeltaPositions(df_train)
df_test = getDeltaPositions(df_test)

#  Output

In [66]:
df_train.to_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))
df_test.to_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))